In [ ]:
#hide
%load_ext autoreload
%autoreload 2

In [1]:
%pip install pybats

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 529.0/529.0 kB 4.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.4/67.4 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.3/89.3 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 6.3 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# default_exp seasonal

# Seasonal Components

> This module contains functions to define the seasonal components in a DGLM. These are *harmonic* seasonal components, meaning they are defined by sine and cosine functions with a specific period. For example, when working with daily time series we often use a weekly seasonal effect of period 7 or an annual seasonal effect of period 365.

In [2]:
#hide
#exporti
import numpy as np
from pybats.forecast import forecast_aR, forecast_R_cov

## Seasonal Components for a DGLM

In [3]:
#export
def seascomp(period, harmComponents):
    p = len(harmComponents)
    n = 2*p
    F = np.zeros([n, 1])
    F[0:n:2] = 1
    G = np.zeros([n, n])

    for j in range(p):
        c = np.cos(2*np.pi*harmComponents[j]/period)
        s = np.sin(2*np.pi*harmComponents[j]/period)
        idx = 2*j
        G[idx:(idx+2), idx:(idx+2)] = np.array([[c, s],[-s, c]])

    return [F, G]

This function is called from `dglm.__init__` to define the seasonal components.

In [4]:
#exporti
def createFourierToSeasonalL(period, harmComponents, Fseas, Gseas):
    p = len(harmComponents)
    L = np.zeros([period, 2*p])
    L[0,:] = Fseas.reshape(-1)
    for i in range(1, period):
        L[i,:] = L[i-1,:] @ Gseas

    return L

In [5]:
#export
def fourierToSeasonal(mod, comp=0):
    phi = mod.L[comp] @ mod.m[mod.iseas[comp]]
    var = mod.L[comp] @ mod.C[np.ix_(mod.iseas[comp], mod.iseas[comp])] @ mod.L[comp].T
    return phi, var

This function transforms the seasonal component of a model from fourier form into more interpretable seasonal components. For example, if `seasPeriods = [7]`, then this would return a vector of length $7$, with each of the seasonal effects.

A simple use case is given below. For a more detailed use of this function, see the following [example](https://github.com/lavinei/pybats_nbdev/blob/master/examples/Poisson_DGLM_In_Depth_Example.ipynb).

In [6]:
import numpy as np
import pandas as pd
from pybats.analysis import analysis
from pybats.shared import load_sales_example2

data = load_sales_example2()

prior_length = 21   # Number of days of data used to set prior

mod = analysis(data.Sales.values, data[['Price', 'Promotion']].values, k=1,
               family='poisson',
               seasPeriods=[7], seasHarmComponents=[[1,2,3]],
               prior_length=prior_length, dates=data.index,
               ret = ['model'])

In [7]:
seas_mean, seas_cov = fourierToSeasonal(mod)

days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
lastday = data.index[-1]

days = [*days[lastday.isoweekday()-1:], *days[:lastday.isoweekday()-1]]

seas_eff = pd.DataFrame({'Day':days,
                         'Effect Mean':np.exp(seas_mean.reshape(-1))})
seas_eff

,Day,Effect Mean
0,Friday,1.002880
1,Saturday,1.349696
2,Sunday,1.325227
3,Monday,0.886239
4,Tuesday,0.837136
5,Wednesday,0.882167
6,Thursday,0.851780


In [8]:
#exporti
def fourierToSeasonalFxnl(L, m, C, iseas):
    phi = L @ m[iseas]
    var = L @ C[np.ix_(iseas, iseas)] @ L.T
    return phi, var

In [9]:
#exporti
def get_seasonal_effect_fxnl(L, m, C, iseas):
    phi, var = fourierToSeasonalFxnl(L, m, C, iseas)
    return phi[0], var[0, 0]

In [10]:
#exporti
def sample_seasonal_effect_fxnl(L, m, C, iseas, delVar, n, nsamps):
    phi_samps = np.zeros([nsamps])
    phi, var = fourierToSeasonalFxnl(L, m, C, iseas)
    phi_samps[:] = phi[0] + np.sqrt(var[0,0])*np.random.standard_t(delVar*n, size = [nsamps])
    return phi_samps

In [11]:
#exporti
def forecast_weekly_seasonal_factor(mod, k, sample = False, nsamps = 1):
    a, R = forecast_aR(mod, k)

    idx = np.where(np.array(mod.seasPeriods) == 7)[0][0]

    if sample:
        return sample_seasonal_effect_fxnl(mod.L[idx], a, R, mod.iseas[idx], mod.delVar, mod.n, nsamps)
    else:
        return get_seasonal_effect_fxnl(mod.L[idx], a, R, mod.iseas[idx])

In [12]:
#exporti
def forecast_path_weekly_seasonal_factor(mod, k, today, period):
    phi_mu = [np.zeros([period]) for h in range(k)]
    phi_sigma = [np.zeros([period, period]) for h in range(k)]
    phi_psi = [np.zeros([period, period, h]) for h in range(1, k)]

    idx = np.where(np.array(mod.seasPeriods) == 7)[0][0]
    L = mod.L[idx]
    iseas = mod.iseas[idx]

    for h in range(k):

        # Get the marginal a, R
        a, R = forecast_aR(mod, h + 1)

        m, v = get_seasonal_effect_fxnl(L, a, R, iseas)
        day = (today + h) % period
        phi_mu[h][day] = m
        phi_sigma[h][day, day] = v
        # phi_mu[h], phi_sigma[h] = get_latent_factor_fxnl_old((today + h) % period, mod.L, a, R, mod.iseas, mod.seasPeriods[0])

        # Find covariances with previous latent factor values
        for j in range(h):
            # Covariance matrix between the state vector at times j, i, i > j
            day_j = (today + j) % period
            cov_jh = forecast_R_cov(mod, j, h)[np.ix_(iseas, iseas)]
            phi_psi[h-1][day, day_j, j] = phi_psi[h-1][day_j, day, j] = (L @ cov_jh @ L.T)[day, day_j]
            # cov_ij = (np.linalg.matrix_power(mod.G, h-j) @ Rlist[j])[np.ix_(mod.iseas, mod.iseas)]

    return phi_mu, phi_sigma, phi_psi